<a href="https://colab.research.google.com/github/ChrisFJardine/ChrisFJardine/blob/main/get_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Map Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup
from decimal import Decimal
import pandas as pd

In [2]:
def getSoup(page):
  soup = BeautifulSoup(page.text, 'html5lib')
  return (soup)


In [3]:
def getRunDate(soup):
  dateString = soup.find('h4').text
  runDate = datetime.strptime(dateString, '%d/%m/%Y').date()
  #print(runDate)
  return(runDate)

In [4]:
def getRunDetails(soup):
  columns = ['Athlete Place','Athlete Name', 'Athlete Gender','Run Time','Avg Pace','Points', 'Run ClassID','Run Length','Run Distance','Run Location','Run Date']
  runDetails = pd.DataFrame(columns = columns)
  course = getCourse(soup).replace('Results','')
  runDate = getRunDate(soup)
  for record in soup.find('table', attrs={'class':'table'}).find_all(['tr']):
    #print(record)
    if record.find('img') is not None:    
      runClassID = record.find('img')['class'][0]
      headerString = record.get_text().split()
      #print(headerString)
      if 'Run' in headerString:
        runType = (headerString[headerString.index('Run') -1])
        for word in headerString:
          if 'km' in word:
            dist  = Decimal(word.replace('km','').replace(')','')) 
    row = [td.text for td in record.find_all(['td'])]
    row.append(runClassID)
    row.append(runType)
    row.append(dist)
    row.append(course)
    row.append(runDate)
    if len(row) == 11:
      runDetails.loc[len(runDetails)] = row
  return(runDetails) 


In [7]:
def getCourse(soup):
  print('xxxx')
  print(soup.find_all('h2'))
  #print(soup)
  if soup.find('h2')is None:
    print('error in getCourse')
    course = 'getCourse Error'
  else:
      course = soup.find('h2').text
  print('course is ')
  print(course)
  return(course)

In [20]:
def get_page(url):
  try:
    page = requests.get(url)
    # print('Request #' + str(i) + ' : retrieved in  ' + str(page.elapsed.total_seconds()) + ' seconds')
  except:
    print('Exception ' + str(page.status_code))  
  if page.status_code == 200:
      print('page status ' + str(page.status_code) + ' ' + url)
      resultPage = BeautifulSoup(page.text, 'html5lib')
      time.sleep(1)
  return(resultPage)

In [23]:
def get_pages(urlbase, startPageNo, pagesToCheck):
  columns = ['Athlete Place','Athlete Name', 'Athlete Gender','Run Time','Avg Pace','Points', 'Run ClassID','Run Length','Run Distance','Run Location','Run Date']
  allResults = pd.DataFrame(columns = columns)
  for i in range (startPageNo, startPageNo + pagesToCheck):          #150 is start of 2022 season. Checks for 50 pages
    url = urlbase + str(i)
    try:
      page = requests.get(url)
     # print('Request #' + str(i) + ' : retrieved in  ' + str(page.elapsed.total_seconds()) + ' seconds')
    except:
      print('Exception ' + str(page.status_code))  
    if page.status_code == 200:
        print('page status ' + str(page.status_code) + ' ' + url)
        resultPage = getSoup(page)
        runDetails = getRunDetails(soup)
        allResults = pd.concat([allResults, runDetails])
        time.sleep(1)
  return(allResults)

In [ ]:
page = requests.get("https://theautomatedclub.com/3CRC/Result/Details/150")
#resultPage = BeautifulSoup(page.text, 'html5lib')
soup = getSoup(page)
#print(page)
runDetails = getRunDetails(soup)


In [24]:
urlbase = "https://theautomatedclub.com/3CRC/Result/Details/"
results = get_pages(urlbase, 2, 3)
display(results)

page status 200 https://theautomatedclub.com/3CRC/Result/Details/2
xxxx
[<h2>Peace Park Results</h2>]
course is 
Peace Park Results
page status 200 https://theautomatedclub.com/3CRC/Result/Details/3
xxxx
[<h2>Peace Park Results</h2>]
course is 
Peace Park Results
page status 200 https://theautomatedclub.com/3CRC/Result/Details/4
xxxx
[<h2>Peace Park Results</h2>]
course is 
Peace Park Results


,Athlete Place,Athlete Name,Athlete Gender,Run Time,Avg Pace,Points,Run ClassID,Run Length,Run Distance,Run Location,Run Date
0,1,Lewis Metcalfe,M,17:45,5:55,50,ig1-0,Short,3,Peace Park,2022-08-31
1,2,Timothy Marsland,M,18:01,6:00,49,ig1-0,Short,3,Peace Park,2022-08-31
2,3,Isla Petersen,F,21:18,7:06,48,ig1-0,Short,3,Peace Park,2022-08-31
3,4,Hazel Metcalfe,F,21:40,7:13,47,ig1-0,Short,3,Peace Park,2022-08-31
4,5,Amy Petersen,F,21:44,7:15,46,ig1-0,Short,3,Peace Park,2022-08-31
...,...,...,...,...,...,...,...,...,...,...,...
113,6,Ian Olsen,M,57:21,5:44,45,ig1-2,Long,10,Peace Park,2022-08-31
114,7,TONY FARNHAM,M,1:00:11,6:01,44,ig1-2,Long,10,Peace Park,2022-08-31
115,8,Chris Jardine,M,1:02:02,6:12,43,ig1-2,Long,10,Peace Park,2022-08-31
116,9,Mitch Bromage,M,1:02:13,6:13,42,ig1-2,Long,10,Peace Park,2022-08-31
